In [1]:
import os
import open3d as o3d
import numpy as np
from PIL import Image
from tqdm import tqdm
import random

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
def change(pos_array):
    p = pos_array
    p_new = np.zeros(p.shape)
    for i in range(len(p)):
        p_new[i] = np.array([-p[i,0],-p[i,2],p[i,1]]) 
    return p_new

In [ ]:
def read_obj(obj_path):
    with open(obj_path) as file:
        points = []
        while 1:
            line = file.readline()
            if not line:
                break
            strs = line.split(" ")
            if strs[0] =='v':
                points.append((float(strs[1]),float(strs[2]),float(strs[3])))

    points = np.array(points)

    
    return points

In [ ]:
if not os.path.exists('./traj'):
    os.makedirs('./traj')
if not os.path.exists('./pic'):
    os.makedirs('./pic')

In [30]:
file_dir = 'D:\\video_dataset\\clouded_leopard_female'
animal_name = file_dir.split('\\')[-1]

In [34]:
point_array = np.array([])
for i in tqdm(range(100)):
    points = change(read_obj(os.path.join(file_dir,'frame_{:06d}.obj'.format(i+1))))
    points = points.reshape(1,-1,3)
#     point_list.append(points)
    if len(point_array)==0:
        point_array = points
    else:
        point_array = np.concatenate((point_array,points),axis=0)


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:26<00:00,  3.81it/s]


In [36]:
select_index = np.array(random.sample(range(point_array.shape[1]), 100))

In [38]:
for i in tqdm(range(100)):
    previous = max(0,i+1-20)
    tmp  = point_array[previous:(i+1),select_index]
    np.save('./traj/{}.npy'.format(i+1),tmp)

100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1569.83it/s]


In [39]:
def test_show(info_index):
    
    line_info = np.load('traj/{}.npy'.format(info_index))
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    
    ctr = vis.get_view_control()
    
    mesh = o3d.io.read_triangle_mesh(os.path.join(file_dir,'frame_{:06d}.obj'.format(info_index)))
    ppp = np.array(mesh.vertices)
    mesh.vertices = o3d.utility.Vector3dVector(change(ppp))
    mesh.paint_uniform_color([1, 0.706, 0])
    vis.add_geometry(mesh)
    vis.update_geometry(mesh)
    for index in range(line_info.shape[1]):
        points = line_info[:,index]
        lines = np.array([[i,i+1] for i in range(len(points)-1)])
        colors = [[1, 0, 0] for i in range(len(lines))]
        line_set = o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector(points),
            lines=o3d.utility.Vector2iVector(lines),
        )
        line_set.colors = o3d.utility.Vector3dVector(colors)
    
        vis.add_geometry(line_set)
        vis.update_geometry(line_set)
    
    information = np.load(os.path.join(file_dir,'info','{:04d}.npz'.format(info_index)))
    intrin = information['intrinsic_mat']
    extrin = np.concatenate( (information['extrinsic_mat'],np.array([0,0,0,1]).reshape((1,4))))
                              
    init_param = ctr.convert_to_pinhole_camera_parameters()
    w, h = 1920,1080
    fx = intrin[0,0]
    fy = intrin[1,1]
    cx = intrin[0,2]
    cy = intrin[1,2]
    init_param.intrinsic.width = w
    init_param.intrinsic.height = h
    
    init_param.intrinsic.set_intrinsics(init_param.intrinsic.width, init_param.intrinsic.height, fx, fy, cx, cy)
    init_param.extrinsic = extrin
    ctr.convert_from_pinhole_camera_parameters(init_param)
    
    vis.poll_events()
    vis.update_renderer()
    
    image = vis.capture_screen_float_buffer(do_render=True)
#     vis.run()
    vis.destroy_window()
    
    image = np.array(image) * 255
    image = image.astype(np.uint8)
    
    return image

In [40]:
for i in tqdm(range(100)):
    image = test_show(i+1)
    img = Image.fromarray(image)
    if not os.path.exists('./pic_{}'.format(animal_name)):
        os.makedirs('./pic_{}'.format(animal_name))
    img.save('./pic_{}/{}.png'.format(animal_name,i+1))


  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

[Open3D WARNING] [SimpleShaderForLineSet] Binding failed with empty geometry::LineSet.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed when preparing data.
[Open3D WARNING] [SimpleShaderForLineSet] Something is wrong in compiling or binding.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed with empty geometry::LineSet.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed when preparing data.
[Open3D WARNING] [SimpleShaderForLineSet] Something is wrong in compiling or binding.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed with empty geometry::LineSet.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed when preparing data.
[Open3D WARNING] [SimpleShaderForLineSet] Something is wrong in compiling or binding.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed with empty geometry::LineSet.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed when preparing data.
[Open3D WARNING] [SimpleShaderForLineSet] Something is wrong in compiling or bindi

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:55<00:00,  1.76s/it]
